In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import bs4
import re
from time import sleep
import os
import pymorphy2
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from tqdm import tqdm
import pickle
from scipy.sparse import csr_matrix
import gensim
from gensim.models import Word2Vec
from multiprocessing import Pool, Lock, Value
import functools
import operator
import requests
import pymorphy2
import nltk
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
morph = pymorphy2.MorphAnalyzer()
from bs4.element import Comment
import os.path
from os import path

In [2]:
stop_words = list(set(stopwords.words('russian')))


In [3]:
queries = pd.read_csv('queries.numerate.txt', sep='\t', header=None)
urls = pd.read_csv('urls.numerate.txt', sep='\t', header=None)
urls.index = urls[1]


In [4]:
docnames = []
for d in os.listdir('content/content'):
    listdocs = os.listdir('content/content' + '/' + d)
    listdocs = [d + '/' + doc_name for doc_name in listdocs]
    docnames.append(listdocs)
docnames = sorted(functools.reduce(operator.iconcat, docnames, []))

In [5]:
if not path.exists('my_data'):
        os.mkdir('my_data')
if not path.exists('my_data/content'):
    os.mkdir('my_data/content')
if not path.exists('my_data/content/content'):
    os.mkdir('my_data/content/content')
list_dir = ['20170702','20170704','20170707','20170708','20170710','20170711','20170717','20170726']
for dir_ in list_dir:
    if not path.exists('my_data/content/content/'+dir_):
        os.mkdir('my_data/content/content/'+dir_)

In [6]:
def easy_tokenizer(text):
    word = str()
    for symbol in text:
        if symbol.isalnum(): word += symbol
        elif word:
            yield word
            word = str()
    if word: yield word

PYMORPHY_CACHE = {}
MORPH = None
#hint, чтобы установка pymorphy2 не была бы обязательной
def get_lemmatizer():
    global MORPH
    if MORPH is None: MORPH = pymorphy2.MorphAnalyzer()
    return MORPH

def pymorphy_tokenizer(text):
    global PYMORPHY_CACHE
    for word in easy_tokenizer(text):
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = get_lemmatizer().parse(word)[0].normal_form          
        yield PYMORPHY_CACHE[word_hash]

In [7]:
PYMORPHY_CACHE = {}
def lemmatizer(words):
    global PYMORPHY_CACHE
    for word in words:
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = morph.parse(word)[0].normal_form
        yield PYMORPHY_CACHE[word_hash]

In [8]:
def find_visible_text(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True
    

def extract_html(soup):
    title = soup.title
    if title is not None:
        title = title.text
        title_words = re.sub(r"[\W+]", " ", title.lower()).split()

    else:
        title_words = [] 
    all_text = soup.findAll(text=True)
    visible_text = filter(find_visible_text, all_text)  
    text =  " ".join(t.strip() for t in visible_text)
    tokens = re.sub(r"[\W+]", " ", text.lower()).split()
 
    title_words = [word for word in title_words if word not in stop_words]
    title_words = list(lemmatizer(title_words))

    words = [word for word in tokens if word not in stop_words]
    words = list(lemmatizer(words))
    
    return (title_words, words)

In [9]:
def parse(file_name):
    with open('content/content/' + file_name, errors='ignore') as read_file:
        html = ' '.join(list(read_file))

    url = html.split()[0]
    soup = BeautifulSoup(' '.join(html.split()[1:]))
    doc_id = urls.at[url, 0]
    res = extract_html(soup)
    
    with open('my_data/content/content/' + file_name + '.txt', 'w+') as f:
        for word in res[0]:
            word +='\n'
            f.write(word)
        f.write('OHMYGODITSSPLIIIIIIIIIIIT\n')
        for word in res[1]:
            word +='\n'
            f.write(word)


In [10]:
for doc in tqdm(docnames):
    parse(doc)

  0%|          | 25/38114 [00:09<4:00:51,  2.64it/s]


KeyboardInterrupt: 

In [11]:
with open('docnames_ordered_by_doc_id.pickle','rb') as f:
    ma = pickle.load(f)

In [12]:
ma_li = list(ma)

In [14]:
words_h = dict()
words_b = dict() #глобальный словарь слов
ind_h = 0
ind_b = 0
y_h_ind = []
x_h_ind = []
x_b_ind = []
y_b_ind = []
data_h = []
data_b = []
i = 0
tot_length = 0
for k in tqdm(ma_li):
    with open('my_data/content/content/'+k+'.txt') as f:     
        terms = list()
        for term in f:    
            if term[:-1] not in stop_words:
                terms.append(term[:-1]) 
        seen_h = set()
        seen_b = set()
        fl = 0
        curs = dict()
        curs_h = dict()
        for term in terms:
            if fl == 0:
                if term == 'OHMYGODITSSPLIIIIIIIIIIIT':
                    fl = 1
                    continue
                if term not in seen_h:
                    seen_h.add(term)
                    curs_h[term] = len(data_h)
                    data_h.append(1)
                    x_h_ind.append(i)
                    if words_h.get(term, -1) == -1:
                        words_h[term] = ind_h
                        ind_h += 1
                    y_h_ind.append(words_h[term])
                else:
                    data_h[curs_h[term]] += 1    
            else :
                    if term not in seen_b:
                        seen_b.add(term)
                        curs[term] = len(data_b)
                        data_b.append(1)
                        x_b_ind.append(i)
                        if words_b.get(term, -1) == -1:
                            words_b[term] = ind_b
                            ind_b += 1
                        y_b_ind.append(words_b[term])
                    else:
                        data_b[curs[term]] += 1
        i += 1    

  1%|          | 219/38114 [00:05<15:26, 40.91it/s]


KeyboardInterrupt: 

In [208]:
class Generator(object):
    def __iter__(self):
        for k in tqdm(ma_li):
            with open('my_data/content/content/'+k+'.txt') as f:
                terms = list()
                for term in f:
                    if term[:-1] not in stop_words:
                        terms.append(term[:-1])
            yield terms

In [209]:
w2v_model = Word2Vec(min_count=19,
                     window=7,
                     size=250,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=6)

In [210]:
gen = Generator()
w2v_model.build_vocab(gen)














  0%|          | 0/38114 [00:00<?, ?it/s]












  0%|          | 1/38114 [00:00<1:09:20,  9.16it/s]












  0%|          | 5/38114 [00:00<56:43, 11.20it/s]  












  0%|          | 6/38114 [00:00<1:05:16,  9.73it/s]












  0%|          | 10/38114 [00:00<54:35, 11.63it/s] 












  0%|          | 19/38114 [00:00<40:19, 15.75it/s]












  0%|          | 25/38114 [00:00<31:46, 19.97it/s]












  0%|          | 30/38114 [00:00<27:23, 23.17it/s]












  0%|          | 34/38114 [00:01<25:59, 24.43it/s]












  0%|          | 40/38114 [00:01<22:14, 28.53it/s]












  0%|          | 48/38114 [00:01<18:38, 34.03it/s]












  0%|          | 59/38114 [00:01<15:09, 41.84it/s]












  0%|          | 65/38114 [00:01<14:58, 42.37it/s]












  0%|          | 78/38114 [00:01<12:00, 52.77it/s]












  0%|          | 86/38114 [00:02<22:34, 28.07it/s]












  0%|          | 92/38114 [00:02<20:40, 30.66it

  2%|▏         | 631/38114 [00:17<07:31, 82.99it/s]












  2%|▏         | 651/38114 [00:17<06:12, 100.49it/s]












  2%|▏         | 668/38114 [00:17<06:16, 99.41it/s] 












  2%|▏         | 686/38114 [00:17<05:28, 113.98it/s]












  2%|▏         | 709/38114 [00:17<04:38, 134.24it/s]












  2%|▏         | 741/38114 [00:17<03:50, 162.31it/s]












  2%|▏         | 763/38114 [00:17<03:40, 169.66it/s]












  2%|▏         | 785/38114 [00:18<06:42, 92.77it/s] 












  2%|▏         | 812/38114 [00:18<05:50, 106.41it/s]












  2%|▏         | 829/38114 [00:19<09:39, 64.34it/s] 












  2%|▏         | 842/38114 [00:19<09:02, 68.67it/s]












  2%|▏         | 854/38114 [00:19<08:22, 74.10it/s]












  2%|▏         | 865/38114 [00:19<08:15, 75.12it/s]












  2%|▏         | 875/38114 [00:19<08:31, 72.85it/s]












  2%|▏         | 884/38114 [00:19<08:12, 75.65it/s]












  2%|▏         | 893/38114 [00:

  4%|▍         | 1439/38114 [00:33<29:31, 20.70it/s]












  4%|▍         | 1443/38114 [00:34<55:25, 11.03it/s]












  4%|▍         | 1448/38114 [00:34<45:21, 13.47it/s]












  4%|▍         | 1451/38114 [00:35<43:42, 13.98it/s]












  4%|▍         | 1457/38114 [00:35<36:56, 16.54it/s]












  4%|▍         | 1460/38114 [00:35<47:53, 12.76it/s]












  4%|▍         | 1464/38114 [00:35<38:29, 15.87it/s]












  4%|▍         | 1469/38114 [00:35<31:31, 19.37it/s]












  4%|▍         | 1472/38114 [00:36<28:56, 21.10it/s]












  4%|▍         | 1475/38114 [00:36<39:59, 15.27it/s]












  4%|▍         | 1482/38114 [00:36<31:32, 19.36it/s]












  4%|▍         | 1486/38114 [00:36<26:51, 22.73it/s]












  4%|▍         | 1490/38114 [00:36<27:32, 22.16it/s]












  4%|▍         | 1493/38114 [00:36<28:51, 21.15it/s]












  4%|▍         | 1501/38114 [00:37<24:27, 24.95it/s]












  4%|▍         | 1505/381

KeyboardInterrupt: 

In [31]:
w2v_model.train(gen, total_examples=w2v_model.corpus_count, epochs=1)

100%|██████████| 38114/38114 [08:48<00:00, 72.10it/s] 


(119957291, 308320688)

In [10]:
sp_h = csr_matrix((data_h, (x_h_ind, y_h_ind)), shape=(i, ind_h),dtype = np.int32) 
sp_b = csr_matrix((data_b, (x_b_ind, y_b_ind)), shape=(i, ind_b),dtype = np.int32)
    

In [11]:
# coding: UTF-8
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import scipy.sparse as sp
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_extraction.text import _document_frequency
#source: https://github.com/arosh/BM25Transformer
class BM25Transformer(BaseEstimator, TransformerMixin):
    """
    Parameters
    ----------
    use_idf : boolean, optional (default=True)
    k1 : float, optional (default=2.0)
    b : float, optional (default=0.75)
    References
    ----------
    Okapi BM25: a non-binary model - Introduction to Information Retrieval
    http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html
    """
    def __init__(self, use_idf=True, k1=2.0, b=0.75):
        self.use_idf = use_idf
        self.k1 = k1
        self.b = b

    def fit(self, X):
        """
        Parameters
        ----------
        X : sparse matrix, [n_samples, n_features]
            document-term matrix
        """
        if not sp.issparse(X):
            X = sp.csc_matrix(X)
        if self.use_idf:
            n_samples, n_features = X.shape
            df = _document_frequency(X)
            idf =   np.log((n_samples - df + 0.5) / (df + 0.5))
            self._idf_diag = sp.spdiags(idf, diags=0, m=n_features, n=n_features)
        return self

    def transform(self, X, copy=True):
        """
        Parameters
        ----------
        X : sparse matrix, [n_samples, n_features]
            document-term matrix
        copy : boolean, optional (default=True)
        """
        if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
            # preserve float family dtype
            X = sp.csr_matrix(X, copy=copy)
        else:
            # convert counts or binary occurrences to floats
            X = sp.csr_matrix(X, dtype=np.float64, copy=copy)

        n_samples, n_features = X.shape

        # Document length (number of terms) in each row
        # Shape is (n_samples, 1)
        dl = X.sum(axis=1)
        # Number of non-zero elements in each row
        # Shape is (n_samples, )
        sz = X.indptr[1:] - X.indptr[0:-1]
        # In each row, repeat `dl` for `sz` times
        # Shape is (sum(sz), )
        # Example
        # -------
        # dl = [4, 5, 6]
        # sz = [1, 2, 3]
        # rep = [4, 5, 5, 6, 6, 6]
        rep = np.repeat(np.asarray(dl), sz)
        # Average document length
        # Scalar value
        avgdl = np.average(dl)
        # Compute BM25 score only for non-zero elements
        data = X.data * (self.k1 + 1) / (X.data + self.k1 * (1 - self.b + self.b * rep / avgdl))
        data[data < 0] = 0
        X = sp.csr_matrix((data, X.indices, X.indptr), shape=X.shape)

        if self.use_idf:
            check_is_fitted(self, '_idf_diag', 'idf vector is not fitted')

            expected_n_features = self._idf_diag.shape[0]
            if n_features != expected_n_features:
                raise ValueError("Input has n_features=%d while the model"
                                 " has been trained with n_features=%d" % (
                                     n_features, expected_n_features))
            # *= doesn't work
            X = X * self._idf_diag

        return X

In [12]:
trans = BM25Transformer()

In [13]:
trans.fit(sp_b)

BM25Transformer(b=0.75, k1=2.0, use_idf=True)

In [14]:
sp_b_t = trans.transform(sp_b)

/home/sargeras/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:50: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.


78662.0


In [15]:
trans.fit(sp_h)

BM25Transformer(b=0.75, k1=2.0, use_idf=True)

In [16]:
sp_h_t = trans.transform(sp_h)

125.0


/home/sargeras/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:50: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.


In [287]:
bad_queries = pd.read_csv('queries.numerate.txt', sep='\t', header=None)
bad_queries = bad_queries.values[:, 1]
urls = pd.read_csv('urls.numerate.txt', sep='\t', header=None)

In [288]:
queries = []
for query in bad_queries:
    reply = requests.get('https://speller.yandex.net/services/spellservice.json/checkText?text=' + query)
    assert(reply.status_code == 200)
    suggest = {change['word']: change['s'][0] for change in reply.json() if len(change['s']) > 0}
    for word, correction in suggest.items():
        query = query.replace(word, correction)
    queries.append(query)

In [289]:
qu_split = []
for g in range(399):
    words = pymorphy_tokenizer(queries[g])
    word = []
    for k in words:
        if k not in stop_words:
            word.append(k) 
    qu_split.append(word)

In [290]:
sample = pd.read_csv('sample.technosphere.ir1.textrelevance.submission.txt')

In [291]:
def res_c1(g):
        yy = np.array(sample[sample['QueryId'] == g+1]['DocumentId'].values.tolist()) - 1

        t =  csr_matrix((len(yy), 1), dtype=np.float64)
        for w in qu_split[g]:
            if words_h.get(w, -1) != -1:
                t += sp_h_t[yy,words_h[w]]
            
            if words_b.get(w, -1) != -1:
                t += sp_b_t[yy,words_b[w]]           
        s = 0
        for w in qu_split[g]:
            if w2v_model.wv.vocab.get(w,-1) == -1:
                s = 1
                break
        if s == 0:
            k_b = 1
            k_h = 1
            for a in w2v_model.wv.most_similar_cosmul(positive=qu_split[g]):
                POS = morph.parse(a[0])[0].tag.POS
                gg = morph.parse(a[0])[0].tag.cyr_repr
                if not ((POS == 'NOUN' or POS is None) and gg.find('имя') == -1 and gg.find('фам') == -1 \
                       and gg.find('гео') == -1):
                     continue
                if a[1] > 0.6:
                    if words_b.get(a[0], -1) != -1:
                        t += (a[1] / k_b) *sp_b_t[yy,words_b[a[0]]]
                        k_b += 1
                        
                    if words_h.get(a[0], -1) != -1:
                        t += (a[1] / k_h) * sp_h_t[yy,words_h[a[0]]]   
                        k_h += 1
             
        t = t.toarray().reshape(-1)
        o = np.argsort(t)[-10:]
        return   yy[o] + 1

In [292]:
mutex = Lock()
n_processed = Value('i', 0)

def func_wrapper(g):
    res = res_c1(g)

    with mutex:
        global n_processed
        n_processed.value += 1
        print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=6) as pool:
    ans = pool.map(func_wrapper, range(399))

399 objects are processed...

In [293]:
for u in range(399):
    ans[u] = ans[u][::-1]

In [294]:
qu_id = []
doc_rel = []

In [296]:
for y in range(399 * 10):
    qu_id.append(y // 10 + 1)
    doc_rel.append(ans[y // 10][y % 10])

In [297]:
result_df = pd.DataFrame({
    'QueryId' : qu_id,
    'DocumentId' : doc_rel
})

In [298]:
result_df.to_csv('result.txt', index=False)